In [15]:
%run common.py

Planning start time: Jul 27, 16:31 (1469626302)


In [16]:
TEST_FOLDER = './test_scenario/'
paths = pd.read_csv(FOLDER + '/mandatory/paths_id.csv', sep=';')

In [17]:
df = pd.read_excel('./input/(Новые) 2.11.xlsx', header=1)
df.dropna(subset=['Номер поезда'], inplace=True)
df.drop(['поезд'], axis=1, inplace=True)
df.columns = ['train_id', 'number', 'ind', 'weight', 'length', 
              'st_first', 'st_end', 'oper_id', 'oper_time', 'oper_st', 'st_from', 'st_to', 'comm']
df['oper_time'] = df.oper_time.apply(pd.to_datetime)
df.drop(['comm'], axis=1, inplace=True)
df['st_first'] = df.st_first.map(stations.drop_duplicates('esr').set_index('esr').station)
df['st_end'] = df.st_end.map(stations.drop_duplicates('esr').set_index('esr').station)
df['oper_st'] = df.oper_st.map(stations.drop_duplicates('esr').set_index('esr').station)
df['st_from'] = df.st_from.map(stations.drop_duplicates('esr').set_index('esr').station)
df['st_to'] = df.st_to.map(stations.drop_duplicates('esr').set_index('esr').station)
df.head()

,train_id,number,ind,weight,length,st_first,st_end,oper_id,oper_time,oper_st,st_from,st_to
7,1002012,2012,8937-201-9244,4000,70,2000036018,2000036796,2,2015-07-29 17:36:00,2000036538,2000036538,2000036518
8,1002014,2014,8937-202-9257,5600,66,2000036018,2000036868,2,2015-07-29 19:06:00,2000036538,2000036538,2000036518
9,1002016,2016,8937-203-9044,4100,71,2000036018,2000036228,2,2015-07-29 20:06:00,2000036538,2000036538,2000036518
10,1002018,2018,8937-204-9271,4000,72,2000036018,2000036932,2,2015-07-29 21:06:00,2000036538,2000036538,2000036518
11,1002020,2020,8937-205-9044,5600,70,2000036018,2000036228,2,2015-07-29 22:06:00,2000036538,2000036538,2000036518


In [18]:
def get_train_info(x):    
    st_from = int(x['oper_st'])
    st_to = int(x['st_end'])    
    route = paths[(paths.ST_FROM == st_from) & (paths.ST_TO == st_to)].ROUTE.values[0].split(',')    
    r_str = ','.join(['station(%s)' % r[:-2] for r in route])    
    return '+train_info(id(%s),info([number(%s),category(%s),weight(%s),length(%s),routes([route([%s])]),joint(-1)]))' %\
     (x.train_id, int(x.number), 2, int(x.weight), int(x.length), r_str)
        
a = df.apply(lambda x: get_train_info(x), axis=1)
a.to_csv(TEST_FOLDER + 'train_info.csv', index=False, sep=';')

In [19]:
import datetime as dt
def get_oper(x):
    ts = int(x.oper_time.timestamp())
    if x.oper_id == 2:        
        s = '+train_depart(id(%s),track(station(%s),station(%s)),time(%s))' % (x.train_id, x.st_from, x.st_to, ts)
    elif x.oper_id == 5:
        s = '+train_ready(id(%s),station(%s),time(%s))' % (x.train_id, x.oper_st, ts)
    else:
        s = '+train_arrive(id(%s),track(station(%s),station(%s)),time(%s))' % (x.train_id, x.st_from, x.st_to, ts)
    return s        
    
a = df.apply(lambda x: get_oper(x), axis=1)
a.to_csv(TEST_FOLDER + 'train_oper.csv', index=False, sep=';')

In [20]:
df_loco = pd.read_excel('./input/(Новые) 2.11.xlsx', header=1, sheetname='ЛОК')
df_loco.dropna(subset=['Борт НОМЕР'], inplace=True)
df_loco.drop(['ЛОК'], axis=1, inplace=True)
#print(df_loco.head())
df_loco.columns = ['loco_id', 'number', 'ser_name', 'depot_st', 'sections', 
              'regions', 'tts', 'dts', 'tr_dts', 'oper', 'oper_time', 'oper_st', 'st_from', 'st_to', 'train_id', 'comm', 'sokr']
df_loco['oper_time'] = df_loco.oper_time.apply(pd.to_datetime)
df_loco.drop(['comm', 'sokr'], axis=1, inplace=True)
df_loco['depot_st'] = df_loco.depot_st.map(stations.drop_duplicates('esr').set_index('esr').station)
df_loco['oper_st'] = df_loco.oper_st.map(stations.drop_duplicates('esr').set_index('esr').station)
df_loco['st_from'] = df_loco.st_from.map(stations.drop_duplicates('esr').set_index('esr').station)
df_loco['st_to'] = df_loco.st_to.map(stations.drop_duplicates('esr').set_index('esr').station)
df_loco.head()

,loco_id,number,ser_name,depot_st,sections,regions,tts,dts,tr_dts,oper,oper_time,oper_st,st_from,st_to,train_id
4,2001502,1502,ВЛ80Р,2000036018,2,С0-С1,40,0,0,64,2015-07-29 16:50:00,2000036518,2000036518,2000036518,0
5,2001503,1503,ВЛ80Р,2000036018,2,С0-С1,40,0,0,64,2015-07-29 17:20:00,2000036518,2000036518,2000036518,0
7,2001505,1505,ВЛ80Р,2000036018,2,С0-С1,40,0,0,5,2015-07-29 17:36:00,2000036538,2000036538,2000036518,1002012
8,2001506,1506,ВЛ80Р,2000036018,2,С0-С1,40,0,0,5,2015-07-29 19:06:00,2000036538,2000036538,2000036518,1002014
9,2001507,1507,ВЛ80Р,2000036018,2,С0-С1,40,0,0,5,2015-07-29 20:06:00,2000036538,2000036538,2000036518,1002016


In [21]:
def get_loco_info(x):
    ser_id = loco_series[loco_series.ser_name == x.ser_name].ser_id.values[0]
    reg_id = ''.join([i for i in x.regions if i.isnumeric()])
    return '+loco_attributes(id(%s),attributes([series(%s),loco_regions([id(%s)]),depot(station(%s)),sections(%s),type(1)]))' %\
    (x.loco_id, ser_id, reg_id, x.depot_st, int(x.sections))
        
a = df_loco.apply(lambda x: get_loco_info(x), axis=1)
a.to_csv(TEST_FOLDER + 'loco_attr.csv', index=False, sep=';')

In [22]:
def get_loco_service(x):
    ts = int(x.oper_time.timestamp())
    return '+fact_loco_next_service(id(%s),fact_time(%s),next_service(dist_to(%s),time_to(%s),type(2001889869)))' %\
    (x.loco_id, ts, 1000, int(x.tts) * 3600)
        
a = df_loco.apply(lambda x: get_loco_service(x), axis=1)
a.to_csv(TEST_FOLDER + 'loco_service.csv', index=False, sep=';')

In [23]:
depot_st_dict = {318803:89370, 319212:90320, 359276:90440, 319201:92000, 319209:92440, 319210:92710, 359271:92570}

In [24]:
df_team = pd.read_excel('./input/(Новые) 2.11.xlsx', header=1, sheetname='ЛБР')
df_team.dropna(subset=['Таб.номер'], inplace=True)
df_team.drop(['ЛБР'], axis=1, inplace=True)
df_team.columns = ['team_id', 'depot_st', 'ready_st', 'name', 'number', 
                  'ser_name', 'long', 'heavy', 'region', 'ready_time', 'oper_id', 'oper_time', 'oper_st', 'st_from', 'st_to',
                  'train_id', 'loco_id', 'depot_ready_time', 'depot_ready_st', 'return_ready_time', 'return_ready_st', 
                  'rest_start_time']
df_team['oper_time'] = df_team.oper_time.apply(pd.to_datetime)
df_team['depot_ready_time'] = df_team.depot_ready_time.apply(lambda x: pd.to_datetime(x) if x !=0 else -1)
df_team['return_ready_time'] = df_team.return_ready_time.apply(lambda x: pd.to_datetime(x) if x !=0 else -1)
df_team['rest_start_time'] = df_team.rest_start_time.apply(lambda x: pd.to_datetime(x) if x !=0 else -1)

df_team['oper_st'] = df_team.oper_st.map(stations.drop_duplicates('esr').set_index('esr').station)
df_team['st_from'] = df_team.st_from.map(stations.drop_duplicates('esr').set_index('esr').station)
df_team['st_to'] = df_team.st_to.map(stations.drop_duplicates('esr').set_index('esr').station)
df_team['depot_ready_st'] = df_team.depot_ready_st.map(stations.drop_duplicates('esr').set_index('esr').station)
df_team['return_ready_st'] = df_team.return_ready_st.map(stations.drop_duplicates('esr').set_index('esr').station)
df_team['depot_ready_st'].fillna(-1, inplace=True)
df_team['return_ready_st'].fillna(-1, inplace=True)

df_team.depot_st.replace(depot_st_dict, inplace=True)
df_team.ready_st.replace(depot_st_dict, inplace=True)
df_team['depot_st'] = df_team.depot_st.map(stations.drop_duplicates('esr').set_index('esr').station)
df_team['ready_st'] = df_team.ready_st.map(stations.drop_duplicates('esr').set_index('esr').station)

df_team.train_id.replace(0, -1, inplace=True)
df_team.loco_id.replace(0, -1, inplace=True)
df_team.head()

,team_id,depot_st,ready_st,name,number,ser_name,long,heavy,region,ready_time,...,oper_st,st_from,st_to,train_id,loco_id,depot_ready_time,depot_ready_st,return_ready_time,return_ready_st,rest_start_time
1,8800002,2000036018,2000036018,МЯСНИКОВ И.И.,8803-000 155,ВЛ80Р,1,1,С0-С1,0,...,2000036518,2000036018,2000036518,-1,-1,2015-07-29 07:30:00,2000036018,2015-07-29 19:00:00,2000036518,2015-07-29 15:10:00
2,8800003,2000036018,2000036018,ГРИЩЕНКО А.В.,8803-000 233,ВЛ80Р,1,1,С0-С1,0,...,2000036518,2000036018,2000036518,-1,-1,2015-07-29 08:30:00,2000036018,2015-07-29 20:00:00,2000036518,2015-07-29 16:10:00
7,8800005,2000036018,2000036018,ШИНКОРЕНКО А.П.,8803-000 622,ВЛ80Р,1,1,С0-С1,2015-07-29 14:00:00,...,2000036538,2000036538,2000036518,1002012,2001505,2015-07-29 14:00:00,2000036018,-1,-1,-1
8,8800006,2000036018,2000036018,ЛЕОНОВИЧ П.А.,8803-000 628,ВЛ80Р,1,1,С0-С1,2015-07-29 15:30:00,...,2000036538,2000036538,2000036518,1002014,2001506,2015-07-29 15:30:00,2000036018,-1,-1,-1
9,8800007,2000036018,2000036018,БЕЗГИНСКИЙ Э.В.,8803-000 735,ВЛ80Р,1,1,С0-С1,2015-07-29 16:30:00,...,2000036538,2000036538,2000036518,1002016,2001507,2015-07-29 16:30:00,2000036018,-1,-1,-1


In [25]:
def get_team_info(x):
    ser_list = x.ser_name.split('; ')
    try:
        ser_id = loco_series[loco_series.ser_name.isin(ser_list)].ser_id.unique()
    except:
        ser_id = -1
    series = ','.join(['id(%s)' % s for s in ser_id])
    reg_id = ''.join([i for i in x.region if i.isnumeric()])
    return '+team_attributes(id(%s),attributes([team_work_regions([id(%s)]),depot(station(%s)),loco_series([%s]),long_train(%s),heavy_train(%s),fake(1),type(1)]))' %\
    (x.team_id, reg_id, x.ready_st, series, int(x.long), int(x.heavy))
        
a = df_team.apply(lambda x: get_team_info(x), axis=1)
a.to_csv(TEST_FOLDER + 'team_attr.csv', index=False, sep=';')

In [26]:
def get_team_ready(x):
    try:
        depot_ts = int(x.depot_ready_time.timestamp()) if x.depot_ready_time != -1 else -1
        return_ts = int(x.return_ready_time.timestamp())  if x.return_ready_time != -1 else -1
        rest_start_ts = int(x.rest_start_time.timestamp())  if x.rest_start_time != -1 else -1
    except:
        depot_ts, return_ts, rest_start_ts = -1, -1, -1
        print(x.team_id, x.depot_ready_time, x.return_ready_time, x.rest_start_time)
    last_ready = 'depot' if depot_ts < return_ts else 'return'
    return '+fact_team_ready(id(%s),ready_depot(time(%s),station(%s)),ready_return(time(%s),station(%s)),last_ready(%s),rest_start_time(%s))' %\
    (x.team_id, x.depot_ready_st, depot_ts, x.return_ready_st, return_ts, last_ready, rest_start_ts)
        
a = df_team.apply(lambda x: get_team_ready(x), axis=1)
a.to_csv(TEST_FOLDER + 'team_ready.csv', index=False, sep=';')

In [27]:
df_team.oper_id.value_counts()

30    149
2      59
35     27
24     13
1      13
Name: oper_id, dtype: int64

In [28]:
states = {0:[33], 1:[2, 3], 2:[28, 30], 3:[31, 54], 4:[37], 5:[24, 26, 43], 6:[1, 42], 7:[34], 8:[35, 38], 9:[25, 41]}

In [29]:
def get_team_location(x):
    oper_time_ts = int(x.oper_time.timestamp())
    try:
        state = [key for key, value in states.items() if x.oper_id in value][0]
    except:
        state = -1
        print(x.oper_id)
        
    if state in [0, 1]:
        s = '+fact_team_location(id(%s),fact_time(%s),location(track(station(%s),station(%s))),oper_time(%s),loco(%s),pass_slot(-1),state(%s))' %\
            (x.team_id, oper_time_ts, x.st_from, x.st_to, oper_time_ts, int(x.loco_id), state)
    else:
        s = '+fact_team_location(id(%s),fact_time(%s),location(station(%s)),oper_time(%s),loco(%s),state(%s))' %\
            (x.team_id, oper_time_ts, x.oper_st, oper_time_ts, int(x.loco_id), state)
    return s
        
a = df_team.apply(lambda x: get_team_location(x), axis=1)
a.to_csv(TEST_FOLDER + 'team_location.csv', index=False, sep=';')

* stations - поменять тяговые плечи
* links - можно оставить
* team_work_region - взять из модельного полигона
* team_region - взять из реального полигона
* slot, slot_pass - взять из реального полигона
* **loco_tonnage - сделать свои**
* **service_stations - сделать свои**

In [58]:
big_st = stations[stations.name.isin(['ЮРТЫ', 'ТАЙШЕТ', 'ЛЕНА', 'ТАКСИМО'])].drop_duplicates('name')
big_st

,station,loco_region,norm_time,name,esr
31,2000036228,2002119307,5796,ТАКСИМО,90440
89,2000036538,2002119322,0,ЮРТЫ,92020
193,2000036518,2002119314,8280,ТАЙШЕТ,92000
432,2000036932,2002119314,6516,ЛЕНА,92710


In [ ]:
paths.columns = [col.lower() for col in paths.columns]
p = paths.dropna().set_index(['st_from', 'st_to'])
p['route'] = p.route.apply(lambda x: [int(float(r)) for r in x.split(',')])

In [54]:
# Юрты - Тайшет
print(p.ix[2000036538, 2000036518].route)

# Тайшет - Юрты
print(p.ix[2000036518, 2000036538].route)


# Тайшет - Лена
print(p.ix[2000036518, 2000036932].route)

# Лена - Тайшет
print(p.ix[2000036932, 2000036518].route)

# Лена - Таксимо
print(p.ix[2000036932, 2000036228].route)

# Таксимо - Лена
print(p.ix[2000036228, 2000036932].route)

[2000036538, 2000036518]
[2000036518, 2000036538]
[2000036518, 2000036784, 2000036792, 2000036796, 2000036800, 2000036894, 2000036902, 2000036904, 2000036868, 2000036872, 2000036932]
[2000036932, 2000036872, 2000036868, 2000036904, 2000036902, 2000036894, 2000036800, 2000036796, 2000036792, 2000036784, 2000036518]
[2000036932, 2000036950, 2000036954, 2000036122, 2000036120, 2000036124, 2000039646, 2000036126, 2000036128, 2000036136, 2000036142, 2000036218, 2000036154, 2000036192, 2000036204, 2000036258, 2000036228]
[2000036228, 2000036258, 2000036204, 2000036192, 2000036154, 2000036218, 2000036142, 2000036136, 2000036128, 2000036126, 2000039646, 2000036124, 2000036120, 2000036122, 2000036954, 2000036950, 2000036932]


In [55]:
stations[stations.name.isin(['ЮРТЫ', 'ТАЙШЕТ', 'ЛЕНА', 'ТАКСИМО'])]
regs = {'ЮРТЫ':['01'], 'ТАЙШЕТ':['01', '14'], 'ЛЕНА':['14', '47'], 'ТАКСИМО':['47']}

,station,loco_region,norm_time,name,esr
31,2000036228,2002119307,5796,ТАКСИМО,90440
32,2000036228,2002119303,5796,ТАКСИМО,90440
33,2000036228,2002119301,5796,ТАКСИМО,90440
34,2000036228,2002119316,5796,ТАКСИМО,90440
89,2000036538,2002119322,0,ЮРТЫ,92020
90,2000036538,2002119299,0,ЮРТЫ,92020
193,2000036518,2002119314,8280,ТАЙШЕТ,92000
194,2000036518,2002119322,8280,ТАЙШЕТ,92000
195,2000036518,2002119301,8280,ТАЙШЕТ,92000
196,2000036518,2002119299,8280,ТАЙШЕТ,92000


In [73]:
def get_station(x):
    s = '+station(id(%s),loco_region(%s),service([]),norm_reserve([norm(weight_type(0),0),norm(weight_type(1),0)]),norm_time(%s))' %\
        (x.station, x.loco_region, x.norm_time)
    return s

# arr = [str(x) for x in p.ix[2000036538, 2000036228].route]
# stations[stations.station.isin(arr)].drop_duplicates('name').sort_values('esr').to_excel('test_stations.xlsx')

st = pd.read_excel('test_stations.xlsx', converters={'loco_region':str})
a = st.apply(get_station, axis=1)
a.to_csv(TEST_FOLDER + 'station.csv', index=False, sep=';')